In [ ]:


import pandas as pd
from scipy import optimize
import math as math
from sklearn import mixture
import numpy as np
import matplotlib as mpl
from gurobipy import *

In [715]:
inst = pd.read_excel("instance_50_4.xlsx")
print(inst)
v = [[0 for k in range(4)] for j in range(50)]
for j in range(50):
    v[j][0] = inst.v1[j]
    v[j][1] = inst.v2[j]
    v[j][2] = inst.v3[j]
    v[j][3] = inst.v4[j]

    Product         r        v1        v2        v3        v4
0         1  0.000000  0.268160  0.671020  0.231280  0.638550
1         2  2.842900  0.763840  0.959650  0.052209  0.384940
2         3  4.353200  0.805510  0.089084  0.901760  0.765700
3         4  9.037600  0.104250  0.797740  0.793290  0.652920
4         5  9.251100  0.469760  0.590780  0.373010  0.381490
5         6  5.052900  0.219060  0.912200  0.832050  0.300020
6         7  6.275800  0.922710  0.101130  0.753830  0.340140
7         8  7.192600  0.320320  0.293290  0.621860  0.918930
8         9  0.239130  0.857540  0.051588  0.394090  0.456270
9        10  5.749300  0.259850  0.504130  0.359280  0.442500
10       11  0.465340  0.878060  0.768380  0.088852  0.454190
11       12  4.225300  0.188270  0.282990  0.341680  0.945280
12       13  4.677300  0.759190  0.225360  0.548670  0.219120
13       14  0.226280  0.031689  0.331290  0.460550  0.882400
14       15  0.650740  0.642340  0.453250  0.645450  0.019875
15      

In [716]:
#1. a
theta = [0.25, 0.25, 0.25, 0.25]
model = Model("MMNLModel")
p = [[0 for k in range(4)] for n in range(50)]

for j in range(50):
    for k in range(4):
        p[j][k] = model.addVar(lb=0, name="p[%s][%s]" % (j, k))

model.update()
x = [0 for row in range(50)]

for j in range(50):
    x[j] = model.addVar(vtype=GRB.BINARY, name="x[%s]" % (j))

model.update()

model.setObjective(quicksum(theta[k] * (quicksum(inst.r[j] * p[j][k] for j in range(1,50))) for k in range(4)), GRB.MAXIMIZE)
model.update()

for k in range(4):
    model.addConstr(quicksum(p[j][k] for j in range(50)),"=",rhs=1)
    
for j in range(1,50):
    for k in range(4):
        model.addConstr((p[j][k]*v[0][k]-v[j][k]*p[0][k]),"<=",rhs=0)

for j in range(1,50):
    for k in range(4):
        model.addConstr(p[j][k]*v[0][k]-v[j][k]*x[j],"<=",rhs=0)
        model.addConstr(p[j][k]*v[0][k]-v[j][k]*p[0][k]+v[j][k]*(1-x[j]),">=",rhs=0)
        model.addConstr(p[j][k],">=",rhs=0)
        
model.update()
model.optimize()

Optimize a model with 788 rows, 250 columns and 1768 nonzeros
Coefficient statistics:
  Matrix range    [2e-04, 1e+00]
  Objective range [7e-03, 2e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [2e-04, 1e+00]
Found heuristic solution: objective -0
Presolve removed 196 rows and 1 columns
Presolve time: 0.01s
Presolved: 592 rows, 249 columns, 1572 nonzeros
Variable types: 200 continuous, 49 integer (49 binary)

Root relaxation: objective 8.171361e+00, 56 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    8.17136    0    3   -0.00000    8.17136      -     -    0s
H    0     0                       8.1140315    8.17136  0.71%     -    0s
     0     0    8.17136    0    4    8.11403    8.17136  0.71%     -    0s
H    0     0                       8.1294797    8.17136  0.52%     -    0s
     0     0    8.17136    0    5    8.12948    8.17136  

In [717]:
#Optimal mix in MMNL model
opt_mix_mmnl=[]
for i in range(49):
    if x[i+1].X == 1:
        opt_mix_mmnl.append(i+1)
        #print(thetamc[i+1].X)
print(opt_mix_mmnl)

[3, 4, 15, 27, 29, 31, 32, 37]


In [718]:
#1. b.
v_new = [[0 for j in range(4)] for k in range(50)]

for j in range(50):
    v_new[j][0] = v[j][0] / sum(inst.v1)           #Adjusting the v values by scaling it.
    v_new[j][1] = v[j][1] / sum(inst.v2)
    v_new[j][2] = v[j][2] / sum(inst.v3)
    v_new[j][3] = v[j][3] / sum(inst.v4)

In [719]:
lam = [0 for j in range(50)]
for j in range(50):
    lam[j] = 0.25 * (v_new[j][0] + v_new[j][1] + v_new[j][2] + v_new[j][3])    #Lambdas in Markov Chain model
print((lam))

[0.018389202288877863, 0.022025264341079497, 0.0261191281090081, 0.023897245259398167, 0.018500427219020649, 0.023081503080778699, 0.021641090104070969, 0.021900636066545041, 0.017955176114836293, 0.015938654670753942, 0.022321551908642982, 0.017838171546126744, 0.01790916948900307, 0.017307239395667091, 0.018014905558290138, 0.022028162703777075, 0.025113176502090696, 0.010526467820748602, 0.028333674697133659, 0.021928705203752345, 0.013436514860484338, 0.013772657647532719, 0.014806871604329259, 0.021318177099543714, 0.030773503903842619, 0.014456309639652502, 0.026271901036685339, 0.023014137808744478, 0.0031663986887674814, 0.020776078869962925, 0.025177476167308147, 0.013737592707144891, 0.019224828791848111, 0.020669181901432559, 0.026809241532867626, 0.021751426461777086, 0.020603357515197034, 0.024131228571475288, 0.019207850515030996, 0.015151100217254555, 0.021785946857524757, 0.023544450218754693, 0.022545220899947056, 0.028915538229684415, 0.017059706063730182, 0.024602363

In [720]:
vnr=[[0 for i in range(50)] for j in range(50)]
nr=0
rho=[[0 for i in range(50)] for j in range(50)]
dr=0
for i in range(1,50):
    for j in range(50):
        nr=0
        dr=0
        if(i!=j):
            for k in range(len(theta)):
                dr=(sum(v[n][k] for n in range(50))-v[i][k])
                nr+=((theta[k]*v[j][k])/dr)
            rho[i][j]=(((nr)-lam[j])/lam[i])
rho_ij=pd.DataFrame(rho)
print(rho_ij)    #Rho values in MC model

          0         1         2         3         4         5         6   \
0   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1   0.021506  0.000000  0.020236  0.022084  0.021321  0.023463  0.019013   
2   0.015876  0.017133  0.000000  0.022479  0.017440  0.020429  0.028041   
3   0.020710  0.020292  0.024461  0.000000  0.019042  0.028857  0.017871   
4   0.019454  0.025082  0.024244  0.024336  0.000000  0.024472  0.020794   
5   0.019545  0.022304  0.022967  0.029914  0.019835  0.000000  0.019056   
6   0.013928  0.019396  0.033745  0.019679  0.018004  0.020344  0.000000   
7   0.019576  0.017616  0.030095  0.026386  0.017563  0.021973  0.021555   
8   0.015268  0.021600  0.033377  0.017411  0.017995  0.016592  0.029975   
9   0.020405  0.023083  0.024223  0.026522  0.019136  0.025173  0.018892   
10  0.020268  0.030604  0.023108  0.020384  0.020620  0.021138  0.021468   
11  0.021845  0.018853  0.028680  0.026854  0.017614  0.020549  0.018557   
12  0.014717

In [721]:
modelmc = Model("MCModel")
thetamc = [0 for i in range(50)]
for j in range(1,50):
    thetamc[j] = modelmc.addVar()

modelmc.update()

modelmc.setObjective(quicksum(thetamc[j] * lam[j] for j in range(1,50)), GRB.MINIMIZE)

modelmc.update()

for j in range(1,50):
    modelmc.addConstr(thetamc[j] - inst.r[j], ">=", rhs=0)
    modelmc.addConstr(thetamc[j] - quicksum(rho[j][k] * thetamc[k] for k in range(1,50)), ">=", rhs=0)

modelmc.update()
modelmc.optimize()

Optimize a model with 98 rows, 49 columns and 2450 nonzeros
Coefficient statistics:
  Matrix range    [2e-03, 1e+00]
  Objective range [3e-03, 3e-02]
  Bounds range    [0e+00, 0e+00]
  RHS range       [3e-02, 1e+01]
Presolve removed 49 rows and 0 columns
Presolve time: 0.01s
Presolved: 49 rows, 49 columns, 2401 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.1530675e+00   2.473899e+01   0.000000e+00      0s
      41    8.1492599e+00   0.000000e+00   0.000000e+00      0s

Solved in 41 iterations and 0.03 seconds
Optimal objective  8.149259896e+00


In [722]:
opt_mix=[]
for i in range(49):
    if thetamc[i+1].X == inst.r[i+1]:
        opt_mix.append(i+1)
print(opt_mix)

[3, 4, 15, 27, 29, 31, 32, 37]


In [723]:
#1. c.
dr_0=0
dr_1=0
dr_2=0
dr_3=0
for i in (opt_mix):
    dr_0+=((v[i][0]))
    dr_1+=((v[i][1]))
    dr_2+=((v[i][2]))
    dr_3+=((v[i][3]))

rev=0
for i in (opt_mix):
    rev+=(inst.r[i]*0.25*v[i][0]/(v[0][0]+dr_0)+(inst.r[i]*0.25*v[i][3]/(v[0][3]+dr_3))+(inst.r[i]*0.25*v[i][1]/(v[0][1]+dr_1))+(inst.r[i]*0.25*v[i][2]/(v[0][2]+dr_2)))
print(rev) #Revenue using formula given

8.15816560986
